# RMDS Workshop 3 Code

The code below is the code used to walkthrough a basic Customer Lifetime Value modeling process as shown by the slides. At the bottom, there is an additional challenge presented to you to model another transaction dataset.

In [ ]:
# Loading Data
import lifetimes
from lifetimes.datasets import load_transaction_data
transaction_data = load_transaction_data()

print(transaction_data.head())

In [ ]:
#Creating an RFM Matrix
from lifetimes.utils import summary_data_from_transaction_data

rfm_matrix = summary_data_from_transaction_data(transaction_data, 'id', 'date')
print(rfm_matrix)

In [ ]:
# Switch Datasets to Include Monetary Value
from lifetimes.datasets import load_cdnow_summary_data_with_monetary_value
rfm_matrix = load_cdnow_summary_data_with_monetary_value()

print(rfm_matrix.head())

In [ ]:
# Fit BG/NBD Model
from lifetimes import BetaGeoFitter
bgf = BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(rfm_matrix['frequency'], rfm_matrix['recency'], rfm_matrix['T'])
print(bgf)

In [ ]:
print(bgf.summary)

In [ ]:
# Assess Model Fit

from lifetimes.plotting import plot_period_transactions
plot_period_transactions(bgf, figsize = (10, 5))

In [ ]:
# Predict Number of Transaction for a Single Customer

t = 10 #predict purchases in 10 periods
individual = rfm_matrix.iloc[20]

print(individual)

In [ ]:
# Predict Number of Purchases in Next 10 Time Periods

prediction = bgf.predict(t, individual['frequency'], individual['recency'], individual['T'])
print(prediction)

In [ ]:
# Checking if Rate and Monetary Value are Independent
returning_customers_summary = rfm_matrix[rfm_matrix['frequency']>0]
returning_customers_summary[['monetary_value', 'frequency']].corr()

In [ ]:
# Fitting Monetary Value Model
from lifetimes import GammaGammaFitter

ggf = GammaGammaFitter(penalizer_coef = 0)
ggf.fit(returning_customers_summary['frequency'],
        returning_customers_summary['monetary_value'])
print(ggf)

In [ ]:
# Predicting Customer Lifetime Value
print(ggf.customer_lifetime_value(
    bgf, #the model to use to predict the number of future transactions
    rfm_matrix['frequency'],
    rfm_matrix['recency'],
    rfm_matrix['T'],
    rfm_matrix['monetary_value'],
    time=12, # months
    discount_rate=0.01 # monthly discount rate ~ 12.7% annually
).head(10))

# Additional Challenge

Using the transaction dataset found [here](https://archive.ics.uci.edu/ml/datasets/Online+Retail+II), fit a Customer Lifetime Value model using the steps we established and report results in the fashion most interesting to you. Note, some data cleaning will be necessary in order to get the dataset into a workable format for LTV analysis. That's part of the fun / learning too.